In [1]:
# Import relevant libraries
import pandas as pd
import time
import re

# Import selenium libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

from datetime import datetime  # Importing the datetime module



In [2]:
start = time.time()

start
# Initialize the WebDriver (ensure the ChromeDriver is in PATH)
driver = webdriver.Chrome()

driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="273373f1e9e2e77a07dd871d362881c3")>

In [3]:
# Search terms
phone_type = ["samsung", "iphone", "infinix-phones-in-kenya",
              "google", "itel", "nokia", 
              "oppo", "oneplus", "realme", 
              "tecno", "vivo", "xiaomi"]

# List to hold all product data
all_product_data = []

In [9]:
def scrape_phones_data(phone_type):
    
    # Base url 
    url = f"https://www.phoneplacekenya.com/product-category/smartphones/{phone_type}/"
    
    driver.get(url)
    
    # Wait for the search results to load
    phones_grid = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "product-wrapper"))        
    )
    
    # Loop through each product element
    for i, product in enumerate(phones_grid):
        try:
            # Re-locate product wrapper in case of stale reference
            phones_grid = WebDriverWait(driver, 20).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "product-wrapper"))        
            )
            
            product = phones_grid[i]  # Access product element by index
            
            # Click on the product
            product.click()
    
            # Wait for the product details to load (single element)
            phone_info = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, "product-images-summary"))
            )
    
            # Product information
            product_name = phone_info.find_element(By.CSS_SELECTOR, ".product_title.entry-title").text
            data = phone_info.find_element(By.CSS_SELECTOR, ".summary.entry-summary").text
    
            # Function to extract information
            def extract_info(text):
                lines = text.strip().split('\n')
                product_info = {}
                for line in lines:
                    if ':' in line:
                        key, value = line.split(':', 1)
                        product_info[key.strip()] = value.strip()
                    else:
                        if 'Reviews' in line:
                            product_info['Reviews'] = line.strip()
                        elif 'IN STOCK' in line:
                            product_info['Status'] = line.strip()
                        elif 'Warranty' in line:
                            product_info['Warranty Info'] = line.strip()
                        elif line.startswith("East Africa") or line.startswith("UAE"):
                            product_info['Region'] = line.strip()
                        elif line.startswith("KSh"):
                            if 'Price 1' not in product_info:
                                product_info['Price 1'] = line.strip()  # First price
                            else:
                                product_info['Price 2'] = line.strip()  # Second price
                return product_info
    
            # Extract the information and append to the list
            product_data = extract_info(data)
            product_data['Product Name'] = product_name  # Add product name to the data
            all_product_data.append(product_data)
    
            time.sleep(2)
    
            # Go back to the previous page
            driver.back()
    
            time.sleep(2)
            
            return product_data
    
        except Exception as e:
            print(f"Error occurred on product {i+1}: {e}")

In [10]:
res = scrape_phones_data('Oppo')

Error occurred on product 1: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6EFFFB125+29573]
	(No symbol) [0x00007FF6EFF6FF50]
	(No symbol) [0x00007FF6EFE2B6EA]
	(No symbol) [0x00007FF6EFE7F815]
	(No symbol) [0x00007FF6EFE7FA6C]
	(No symbol) [0x00007FF6EFECB917]
	(No symbol) [0x00007FF6EFEA733F]
	(No symbol) [0x00007FF6EFEC86BC]
	(No symbol) [0x00007FF6EFEA70A3]
	(No symbol) [0x00007FF6EFE712DF]
	(No symbol) [0x00007FF6EFE72441]
	GetHandleVerifier [0x00007FF6F032C76D+3377613]
	GetHandleVerifier [0x00007FF6F0377B67+3685831]
	GetHandleVerifier [0x00007FF6F036CF8B+3641835]
	GetHandleVerifier [0x00007FF6F00BB2A6+816390]
	(No symbol) [0x00007FF6EFF7B25F]
	(No symbol) [0x00007FF6EFF77084]
	(No symbol) [0x00007FF6EFF77220]
	(No symbol) [0x00007FF6EFF6607F]
	BaseThreadInitThunk [0x00007FF9A9A47374+20]
	RtlUserThreadStart [0x00007FF9AA9DCC91+33]

Error occurred on product 2: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6EFFFB125+29573]
	(No symbol) [0x00007FF6EFF6FF50]
	(No symbol) [0

In [11]:
res

{'Reviews': '0 Reviews',
 'Status': 'IN STOCK',
 'Price 1': 'KSh 55,000',
 'Price 2': 'KSh 44,500',
 'RAM': '8GB, 12GB',
 'Storage': '256GB, 512GB',
 'OS': 'Android 14, ColorOS 14',
 'Camera': '50MP + 8MP + 2MP',
 'Selfie': '32MP',
 'Display': '6.67 inch, AMOLED,120Hz',
 'Processor': 'Mediatek Dimensity 6300 (6 nm)',
 'Connectivity': 'Dual sim, 3G, 4G, 5G',
 'Colors': 'Olive Green, Amber Orange',
 'Battery': '5000 mAh, 45W wired, PD2.0',
 'Brands': 'Oppo',
 'Tags': '',
 'Product Name': 'Oppo Reno 12F 5G'}

In [14]:
all_product_data

[{'Reviews': '0 Reviews',
  'Status': 'IN STOCK',
  'RAM': '4GB',
  'Internal Memory': '64GB,128GB',
  'Display': '6.67 inches, IPS LCD',
  'OS': 'Android 14, ColorOS 14',
  'Chipset': 'Octa-core, Qualcomm Snapdragon 6s Gen 1',
  'Main Camera': '8MP + Auxiliary Lens',
  'Selfie Camera': '5 MP, f/2.2, (wide)',
  'Connectivity': '4G, Bluetooth 5.0, GPS, NFC, USB Type-C',
  'Battery': '5,100mAh, 45W',
  'Price 1': 'KSh 17,500',
  'Price 2': 'KSh 15,000',
  'SKU': 'N/A',
  'Brands': 'Oppo',
  'Product Name': 'Oppo A3x 4G'},
 {'Reviews': '0 Reviews',
  'Status': 'IN STOCK',
  'RAM': '6GB',
  'Internal Memory': '128GB',
  'Display': '67 inches, IPS LCD',
  'OS': 'Android 14, ColorOS 14',
  'Chipset': 'Octa-core, Qualcomm Snapdragon 6s Gen 1',
  'Main Camera': '50MP + Auxiliary Lens',
  'Selfie Camera': '5 MP, f/2.2, (wide)',
  'Connectivity': '4G, Bluetooth 5.0, GPS, NFC, USB Type-C',
  'Battery': '5,100mAh, 45W',
  'Price 1': 'KSh 26,000',
  'Price 2': 'KSh 24,000',
  'SKU': 'N/A',
  'Brand

In [3]:



    

def scrape_phones_data(phone_type):
    # Base URL 
    url = f"https://www.phoneplacekenya.com/product-category/smartphones/{phone_type}/"
    
    driver.get(url)
    
    # Wait for the search results to load
    phones_grid = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "product-wrapper"))        
    )
    
    # Loop through each product element
    for i, product in enumerate(phones_grid):
        try:
            # Re-locate product wrapper in case of stale reference
            phones_grid = WebDriverWait(driver, 20).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "product-wrapper"))        
            )
            
            product = phones_grid[i]  # Access product element by index
            
            # Click on the product
            product.click()
    
            # Wait for the product details to load (single element)
            phone_info = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, "product-images-summary"))
            )
    
            # Product information
            product_name = phone_info.find_element(By.CSS_SELECTOR, ".product_title.entry-title").text
            data = phone_info.find_element(By.CSS_SELECTOR, ".summary.entry-summary").text
    
            # Function to extract information
            def extract_info(text):
                lines = text.strip().split('\n')
                product_info = {}
                for line in lines:
                    if ':' in line:
                        key, value = line.split(':', 1)
                        product_info[key.strip()] = value.strip()
                    else:
                        if 'Reviews' in line:
                            product_info['Reviews'] = line.strip()
                        elif 'IN STOCK' in line:
                            product_info['Status'] = line.strip()
                        elif 'Warranty' in line:
                            product_info['Warranty Info'] = line.strip()
                        elif line.startswith("East Africa") or line.startswith("UAE"):
                            product_info['Region'] = line.strip()
                        elif line.startswith("KSh"):
                            if 'Price 1' not in product_info:
                                product_info['Price 1'] = line.strip()  # First price
                            else:
                                product_info['Price 2'] = line.strip()  # Second price
                return product_info
    
            # Extract the information and append to the list
            product_data = extract_info(data)
            product_data['Product Name'] = product_name  # Add product name to the data
            
            # Add the current date and time to the product data
            product_data['Scrape Date'] = datetime.now().strftime("%Y-%m-%d")  # Add the date (Year-Month-Day)
            product_data['Scrape Time'] = datetime.now().strftime("%H:%M:%S")  # Add the time (Hour:Minute:Second)
            
            all_product_data.append(product_data)
    
            time.sleep(2)
    
            # Go back to the previous page
            driver.back()
    
            time.sleep(2)
    
        except Exception as e:
            print(f"Error occurred on product {i+1}: {e}")

    # Create a DataFrame with all products after scraping
    df = pd.DataFrame(all_product_data)
    
    # # Step 1: Keep only the first 20 columns (if applicable)
    # df_limited = df.iloc[:, :20]  # Select columns from the first up to the 20th (0-based index)

    # # Step 2: Move the 20th column (index 19) to the first position
    # columns = df_limited.columns.tolist()  # Get all column names as a list
    # column_20 = columns.pop(19)  # Remove the 20th column from its current position
    # column_18 = columns.pop(18)
    # columns.insert(0, column_20)  # Insert the 20th column at the start
    # columns.insert(1, column_18)

    # # Reorder the DataFrame with the 20th column at the first position
    # df_reordered = df_limited[columns]
    
    # List of required columns in the specific order
    ordered_columns = ['Scrape Date', 'Scrape Time', 'Product Name']
    
    # Add other columns except 'Price 1' and 'Price 2'
    other_columns = [col for col in df.columns if col not in ['Scrape Date', 'Scrape Time', 'Product Name', 'Price 1', 'Price 2']]
    
    # Add 'Price 1' and 'Price 2' at the end
    ordered_columns += other_columns + ['Price 1', 'Price 2']
    
    # Reorder the DataFrame
    df_reordered = df[ordered_columns]
    return df_reordered
    
   




In [4]:
phone_type[6]

'oppo'

In [5]:
df = scrape_phones_data(phone_type[6])

Error occurred on product 1: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7ABDAB125+29573]
	(No symbol) [0x00007FF7ABD1FF50]
	(No symbol) [0x00007FF7ABBDB6EA]
	(No symbol) [0x00007FF7ABC2F815]
	(No symbol) [0x00007FF7ABC2FA6C]
	(No symbol) [0x00007FF7ABC7B917]
	(No symbol) [0x00007FF7ABC5733F]
	(No symbol) [0x00007FF7ABC786BC]
	(No symbol) [0x00007FF7ABC570A3]
	(No symbol) [0x00007FF7ABC212DF]
	(No symbol) [0x00007FF7ABC22441]
	GetHandleVerifier [0x00007FF7AC0DC76D+3377613]
	GetHandleVerifier [0x00007FF7AC127B67+3685831]
	GetHandleVerifier [0x00007FF7AC11CF8B+3641835]
	GetHandleVerifier [0x00007FF7ABE6B2A6+816390]
	(No symbol) [0x00007FF7ABD2B25F]
	(No symbol) [0x00007FF7ABD27084]
	(No symbol) [0x00007FF7ABD27220]
	(No symbol) [0x00007FF7ABD1607F]
	BaseThreadInitThunk [0x00007FF9A9A47374+20]
	RtlUserThreadStart [0x00007FF9AA9DCC91+33]

Error occurred on product 2: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7ABDAB125+29573]
	(No symbol) [0x00007FF7ABD1FF50]
	(No symbol) [0

In [6]:
df

,Scrape Date,Scrape Time,Product Name,Reviews,Status,RAM,Storage,Battery,Main Camera,Selfie Camera,...,Colours,OS,Camera,Selfie,Colors,Warranty Info,Region,SKU,Price 1,Price 2
0,2024-09-30,12:05:04,Oppo A58 4G,0 Reviews,IN STOCK,8GB,128GB,"5,000 mAh with 33W Fast Charging",50MP + 2MP,8MP,...,"Glowing Black, Dazzling Green","Android 13, ColorOS 13.1",NaN,NaN,NaN,NaN,NaN,NaN,"KSh 27,000",NaN
1,2024-09-30,12:05:11,Oppo A98 5G,0 Reviews,IN STOCK,8GB,256GB,"5,000 mAh",NaN,NaN,...,NaN,"Android 13, ColorOS 13.1",64MP + 2MP+ 2MP,32MP,"Cool Black, Dreamy Blue, Bold Gold",NaN,NaN,NaN,"KSh 45,500","KSh 39,500"
2,2024-09-30,12:05:18,Oppo A78 4G,0 Reviews,IN STOCK,8GB,256GB,"5,000 mAh, 67W",NaN,NaN,...,NaN,"Android 13, ColorOS 13.1",50MP + 2MP,8MP,"Mist Black, Aqua Green",NaN,NaN,NaN,"KSh 35,000","KSh 32,500"
3,2024-09-30,12:05:28,Oppo A38,0 Reviews,IN STOCK,4/6GB,128GB,5000 mAh,NaN,NaN,...,NaN,"Android 13, ColorOS 13.1",50MP + 2MP,5MP,"Glowing Black, Glowing Gold",No Local Warranty,UAE / Dubai,N/A,"KSh 19,500","KSh 18,500"
4,2024-09-30,12:05:37,Oppo Reno 10 5G,0 Reviews,IN STOCK,8GB,"128GB, 256GB","5000 mAh, 67W",NaN,NaN,...,NaN,"Android 13, ColorOS 13",64MP + 32MP + 8MP,32MP,"Silvery Grey, Ice Blue",NaN,NaN,NaN,"KSh 58,000","KSh 55,000"
5,2024-09-30,12:05:43,Oppo Reno 10 Pro 5G,0 Reviews,IN STOCK,12GB,256GB,"4,600 mAh, 80W",NaN,NaN,...,NaN,"Android 13, ColorOS 13",50MP + 32MP + 8MP,32MP,"Silvery Grey, Glossy Purple",NaN,NaN,NaN,"KSh 68,000","KSh 65,000"
6,2024-09-30,12:05:50,Oppo A58,0 Reviews,IN STOCK,"6GB, 8GB",128GB,"5000 mAh, 33W",50MP + 2MP,NaN,...,NaN,"Android 13, ColorOS 13.1",NaN,8MP,"Glowing Black, Dazzling Green",NaN,NaN,N/A,"KSh 28,000","KSh 27,000"


In [ ]:
# Iterate over each search term and collect all data
all_reordered_data = []
for phone in phone_type:
    print(f"Scraping data for: {phone}")
    reordered_data = scrape_phones_data(phone)
    all_reordered_data.append(reordered_data)  # Append the result for each phone type
    print(f"Finished scraping for: {phone}") 

# Close the driver
driver.quit()

# Concatenate all the reordered DataFrames into one if necessary
final_df = pd.concat(all_reordered_data, ignore_index=True)

# Optionally, save the final DataFrame to a CSV
final_df.to_csv('../data/scraped_phones_with_datetime.csv', index=False)

# Display the updated DataFrame
print(final_df.shape)
print("\n *************************************")
print(final_df.info())
# Display the first 10 rows of the DataFrame
print(final_df.head(10))